In [ ]:
import keras
import datetime
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import math
import h5py

from tensorflow.keras import Model, Sequential

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Input, Activation, Concatenate, Convolution2D, GlobalAveragePooling2D, Layer, Add, MaxPool2D

from keras.callbacks import TensorBoard

from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers

import warnings
warnings.simplefilter('ignore')



We import the training and testing data

In [ ]:
train_dir = 'imagenet2/train'
val_dir = 'imagenet2/val'

batch_size=64
def random_crop_preprocessing(img):
    crop_size = 227
    h, w, _ = img.shape
    top = np.random.randint(0, h - crop_size + 1)
    left = np.random.randint(0, w - crop_size + 1)
    cropped = img[top:top+crop_size, left:left+crop_size, :]
    mean = np.array([104, 117, 123], dtype=np.float32)
    cropped = cropped - mean
    return cropped


train_datagen = ImageDataGenerator(
    rescale=1.,
    horizontal_flip=True,
    preprocessing_function=random_crop_preprocessing
)

def center_crop_preprocessing(img):
    crop_size = 227
    h, w, _ = img.shape
    top = (h - crop_size) // 2
    left = (w - crop_size) // 2
    cropped = img[top:top+crop_size, left:left+crop_size, :]
    mean = np.array([104, 117, 123], dtype=np.float32)
    cropped = cropped - mean
    return cropped

val_datagen = ImageDataGenerator(
    rescale=1.,
    preprocessing_function=center_crop_preprocessing
)



train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(227, 227),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(227, 227),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator1 = val_datagen.flow_from_directory(
    'imagenet2/val',
    target_size=(227, 227),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.


We define the Squeezenet class allowing us to adjust the ratio of 3x3 filters

In [ ]:
one = (1, 1)
two = (2, 2)
three = (3, 3)
five = (5, 5)
seven = (7, 7)
thirteen = (13, 13)

input_shape = (227, 227, 3)
class FireModule(object):
    def __init__(self, layer_number: int, activation: str, kernel_initializer: str,sq=0.125,p3=0.5) -> None:
        self.layer_number = layer_number
        self.activation = activation
        self.kernel_initializer = kernel_initializer
        self.sq=sq
        self.p3=p3

    def build_module(self, fire_input: Layer) -> Layer:
        global one, three, five

        output_size = 128 * (1 + (self.layer_number//2))

        squeeze_1x1_filters = int(output_size*self.sq)
        expand_1x1_filters = int(output_size*(1-self.p3))
        expand_3x3_filters= int(output_size*self.p3)

        squeeze_1x1 = Conv2D(name=f'fire_{self.layer_number+2}_squeeze_1x1',
            filters=squeeze_1x1_filters, kernel_size=one, strides=1, padding='valid', activation=self.activation,
            kernel_initializer=self.kernel_initializer)(fire_input)
        expand_1x1 = Conv2D(name=f'fire_{self.layer_number+2}_expand_1x1',
            filters=expand_1x1_filters, kernel_size=one, strides=1, padding='valid', activation=self.activation,
            kernel_initializer=self.kernel_initializer)(squeeze_1x1)
        expand_3x3 = Conv2D(name=f'fire_{self.layer_number+2}_expand_3x3',
            filters=expand_3x3_filters, kernel_size=three, strides=1, padding='same', activation=self.activation,
            kernel_initializer=self.kernel_initializer)(squeeze_1x1)

        fire = Concatenate(name=f'fire_{self.layer_number+2}')([expand_1x1, expand_3x3])

        return fire
class SqueezeNet(object):

    def __init__(self, activation: str='relu', kernel_initializer: str='glorot_uniform', squeeze_ratio=0.125, p3=0.5) -> None:
        self.activation = activation
        self.kernel_initializer = kernel_initializer
        self.sq=squeeze_ratio
        self.p3=p3

    def vanilla_model(self, input_shape: tuple=(224, 224, 3), n_classes: int=1000) -> None:

        inp = Input(shape=input_shape, name='Input')

        conv_1 = Conv2D(name="Conv_1",
            filters=96, kernel_size=seven, strides=2, padding='same', activation=self.activation, kernel_initializer=self.kernel_initializer)(inp)
        maxpool_1 = MaxPool2D(name="MaxPool_1",
            pool_size=three, strides=2)(conv_1)

        fire_2 = FireModule(layer_number=0, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(maxpool_1)
        fire_3 = FireModule(layer_number=1, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(fire_2)
        fire_4 = FireModule(layer_number=2, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(fire_3)

        maxpool_2 = MaxPool2D(name="MaxPool_2",
            pool_size=three, strides=2)(fire_4)

        fire_5 = FireModule(layer_number=3, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(maxpool_2)
        fire_6 = FireModule(layer_number=4, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(fire_5)
        fire_7 = FireModule(layer_number=5, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(fire_6)
        fire_8 = FireModule(layer_number=6, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(fire_7)

        maxpool_3 = MaxPool2D(name="MaxPool_3",
            pool_size=three, strides=2)(fire_8)

        fire_9 = FireModule(layer_number=7, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(maxpool_3)

        dropout = Dropout(0.5, name="Dropout")(fire_9)

        conv_10 = Conv2D(name="Conv_10",
            filters=n_classes, kernel_size=one, strides=1, padding='valid', activation=self.activation, kernel_initializer=self.kernel_initializer)(dropout)
        gap_11 = GlobalAveragePooling2D()(conv_10)
        out_put=Activation('softmax',name='softmax')(gap_11)


        self.model = Model(inputs=inp, outputs=out_put)


    def bypass_model(self, input_shape: tuple=(224, 224, 3), n_classes: int=1000) -> None:

        inp = Input(shape=input_shape, name='Input')

        conv_1 = Conv2D(name="Conv_1",
            filters=96, kernel_size=seven, strides=2, padding='same', activation=self.activation, kernel_initializer=self.kernel_initializer)(inp)
        maxpool_1 = MaxPool2D(name="MaxPool_1",
            pool_size=three, strides=2)(conv_1)

        fire_2 = FireModule(layer_number=0, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(maxpool_1)
        fire_3 = FireModule(layer_number=1, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(fire_2)
        bypass_1 = Add(name="Bypass_1")([fire_2, fire_3])
        fire_4 = FireModule(layer_number=2, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(bypass_1)

        maxpool_2 = MaxPool2D(name="MaxPool_2",
            pool_size=three, strides=2)(fire_4)

        fire_5 = FireModule(layer_number=3, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(maxpool_2)
        bypass_2 = Add(name="Bypass_2")([maxpool_2, fire_5])
        fire_6 = FireModule(layer_number=4, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(bypass_2)
        fire_7 = FireModule(layer_number=5, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(fire_6)
        bypass_3 = Add(name="Bypass_3")([fire_6, fire_7])
        fire_8 = FireModule(layer_number=6, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(bypass_3)

        maxpool_3 = MaxPool2D(name="MaxPool_3",
            pool_size=three, strides=2)(fire_8)

        fire_9 = FireModule(layer_number=7, activation=self.activation, kernel_initializer=self.kernel_initializer,sq=self.sq,p3=self.p3).build_module(maxpool_3)
        bypass_4 = Add(name="Bypass_4")([maxpool_3, fire_9])

        dropout = Dropout(0.5, name="Dropout")(bypass_4)

        conv_10 = Conv2D(name="Conv_10",
            filters=n_classes, kernel_size=one, strides=1, padding='valid', activation=self.activation, kernel_initializer=self.kernel_initializer)(dropout)
        gap_11 = GlobalAveragePooling2D()(conv_10)
        out_put=Activation('softmax',name='softmax')(gap_11)

        self.model = Model(inputs=inp, outputs=out_put)


    def build_model(self, input_shape: tuple=(224, 224, 3), n_classes: int=1000, choice: str='vanilla') -> Model:

        if choice == "vanilla":
            self.vanilla_model(input_shape, n_classes)
        else:
            self.bypass_model(input_shape, n_classes)

        return self.model

In [ ]:
p3_history={}
p3_model_dic={}
base_lr=0.0003
batch_size=64
steps_per_epoch = train_generator.samples // batch_size

We vary the raio of 3x3 filters and train the corresponding models

In [ ]:
ll=[0.01,0.125,0.25,0.375, 0.5, 0.625, 0.75, 0.875, 0.99]
l=len(ll)
for i in range(l):
    p=ll[i]
    total_epochs = 60
    base_lr=0.0003

    def poly_lr_scheduler(epoch, lr):
        decay = 1.0 - (epoch / total_epochs)
        new_lr = base_lr * decay
        return new_lr

    lr_callback = LearningRateScheduler(poly_lr_scheduler)
    tensorboard_callback = TensorBoard(log_dir='./logs')
    checkpoint_callback = ModelCheckpoint(
        filepath="SqueezeNet_p3x3_ratio_{}_epoch_{{epoch:02d}}.h5".format(p),
        save_freq='epoch',
        verbose=1
    )
    print('p is', p)
    sqnet = SqueezeNet(p3=p)
    model =sqnet.build_model(input_shape=input_shape, n_classes=10, choice='bypass')
    model.compile(optimizer= tf.keras.optimizers.Adam(base_lr),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

    training_history = model.fit(
        train_generator,
        epochs=total_epochs,
        steps_per_epoch=steps_per_epoch,
        validation_data=val_generator,
        validation_steps=val_generator.samples // batch_size,
        callbacks=[lr_callback, checkpoint_callback, tensorboard_callback],
        verbose=1
    )
    print(f'p3 ratio is, {p}', model.summary())
    model.save_weights(f'Squeeze_net_p3x3_{p}_new.weights.h5')
    val_generator1 = val_datagen.flow_from_directory(
        'imagenet2/val',
        target_size=(227, 227),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    val_steps = val_generator1.samples // batch_size

    predictions = model.predict(val_generator1, steps=val_steps)
    true_labels = val_generator1.classes[:val_steps * batch_size]

    num_classes = len(val_generator1.class_indices)
    true_labels_one_hot = tf.keras.utils.to_categorical(true_labels, num_classes)

    top1_metric = tf.keras.metrics.CategoricalAccuracy()
    top5_metric = tf.keras.metrics.TopKCategoricalAccuracy(k=5)
    top1_metric.update_state(true_labels_one_hot, predictions)
    top5_metric.update_state(true_labels_one_hot, predictions)

    print(f"Top-1 Accuracy of squeezenet_{p}", top1_metric.result().numpy())
    print(f"Top-5 Accuracy of squeezenet_{p}", top5_metric.result().numpy())
    p3_history[p]=training_history
    p3_model_dic[p]=model
    print('save model', p)


p is 0.01
Epoch 1/60
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1634 - loss: 2.2258
Epoch 1: saving model to SqueezeNet_p3x3_ratio_0.01_epoch_01.h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 186s 1s/step - accuracy: 0.1639 - loss: 2.2248 - val_accuracy: 0.3783 - val_loss: 1.7987 - learning_rate: 3.0000e-04
Epoch 2/60
  1/147 ━━━━━━━━━━━━━━━━━━━━ 2:55 1s/step - accuracy: 0.3125 - loss: 2.0571
Epoch 2: saving model to SqueezeNet_p3x3_ratio_0.01_epoch_02.h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 21s 135ms/step - accuracy: 0.3125 - loss: 2.0571 - val_accuracy: 0.3791 - val_loss: 1.8008 - learning_rate: 2.9500e-04
Epoch 3/60
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3742 - loss: 1.7895
Epoch 3: saving model to SqueezeNet_p3x3_ratio_0.01_epoch_03.h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 217s 1s/step - accuracy: 0.3743 - loss: 1.7893 - val_accuracy: 0.4444 - val_loss: 1.6580 - learning_rate: 2.9000e-04
Epoch 4/60
  1/147 ━━━━━━━━━━━━━━━━━━━━ 3:40 2s/step - accuracy: 0.4219 - loss: 1.7487
Epoch 4: saving model to SqueezeNet_p3x3_ratio_0.01_epoch_04.h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 26s 167ms/step - accuracy: 0.4219 - loss: 1.7487 - val_accuracy: 0.4237 - val_loss: 1.6906 - learning_rate: 2.8500e-04
Epoch 5/60
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4331 - loss: 1.6534
Epoch 5: saving model to SqueezeNet_p3x3_ratio_0.01_epoch_05.h5


...

147/147 ━━━━━━━━━━━━━━━━━━━━ 47s 300ms/step - accuracy: 0.9062 - loss: 0.3839 - val_accuracy: 0.7974 - val_loss: 0.7436 - learning_rate: 2.5000e-05
Epoch 57/60
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9146 - loss: 0.2739
Epoch 57: saving model to SqueezeNet_p3x3_ratio_0.99_epoch_57.h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 396s 3s/step - accuracy: 0.9146 - loss: 0.2739 - val_accuracy: 0.7999 - val_loss: 0.7419 - learning_rate: 2.0000e-05
Epoch 58/60
  1/147 ━━━━━━━━━━━━━━━━━━━━ 6:06 3s/step - accuracy: 0.9375 - loss: 0.1774
Epoch 58: saving model to SqueezeNet_p3x3_ratio_0.99_epoch_58.h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 46s 298ms/step - accuracy: 0.9375 - loss: 0.1774 - val_accuracy: 0.7984 - val_loss: 0.7430 - learning_rate: 1.5000e-05
Epoch 59/60
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9201 - loss: 0.2494
Epoch 59: saving model to SqueezeNet_p3x3_ratio_0.99_epoch_59.h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 406s 3s/step - accuracy: 0.9201 - loss: 0.2494 - val_accuracy: 0.7961 - val_loss: 0.7551 - learning_rate: 1.0000e-05
Epoch 60/60
  1/147 ━━━━━━━━━━━━━━━━━━━━ 6:03 2s/step - accuracy: 0.9219 - loss: 0.4097
Epoch 60: saving model to SqueezeNet_p3x3_ratio_0.99_epoch_60.h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 49s 321ms/step - accuracy: 0.9219 - loss: 0.4097 - val_accuracy: 0.7959 - val_loss: 0.7547 - learning_rate: 5.0000e-06
Found 3925 images belonging to 10 classes.
61/61 ━━━━━━━━━━━━━━━━━━━━ 46s 757ms/step
Top-1 Accuracy of squeezenet_0.99 0.7973873
Top-5 Accuracy of squeezenet_0.99 0.9764344
save model 0.99


We check the accuracy

In [ ]:
ll=[0.01,0.125,0.25,0.375, 0.5, 0.625, 0.75, 0.875, 0.99]
l=len(ll)
for i in range(l):
    print("\n")
    print("percentage of 3x3 filters is ", ll[i])
    p=ll[i]
    sqnet = SqueezeNet(p3=p)
    model =sqnet.build_model(input_shape=input_shape, n_classes=10, choice='bypass')
    model.load_weights(f'Squeeze_net_p3x3_{p}_new.weights.h5')
    val_steps = val_generator1.samples // batch_size

    predictions = model.predict(val_generator1, steps=val_steps)
    true_labels = val_generator1.classes[:val_steps * batch_size]

    num_classes = len(val_generator1.class_indices)
    true_labels_one_hot = tf.keras.utils.to_categorical(true_labels, num_classes)

    top1_metric = tf.keras.metrics.CategoricalAccuracy()
    top5_metric = tf.keras.metrics.TopKCategoricalAccuracy(k=5)
    top1_metric.update_state(true_labels_one_hot, predictions)
    top5_metric.update_state(true_labels_one_hot, predictions)

    print(f"Top-1 Accuracy of squeezenet_{p}", top1_metric.result().numpy())
    print(f"Top-5 Accuracy of squeezenet_{p}", top5_metric.result().numpy())




percentage of 3x3 filters is  0.01
61/61 ━━━━━━━━━━━━━━━━━━━━ 21s 335ms/step
Top-1 Accuracy of squeezenet_0.01 0.7320697
Top-5 Accuracy of squeezenet_0.01 0.9667008


percentage of 3x3 filters is  0.125
61/61 ━━━━━━━━━━━━━━━━━━━━ 22s 352ms/step
Top-1 Accuracy of squeezenet_0.125 0.7753586
Top-5 Accuracy of squeezenet_0.125 0.97284836


percentage of 3x3 filters is  0.25
61/61 ━━━━━━━━━━━━━━━━━━━━ 23s 382ms/step
Top-1 Accuracy of squeezenet_0.25 0.79431355
Top-5 Accuracy of squeezenet_0.25 0.97873974


percentage of 3x3 filters is  0.375
61/61 ━━━━━━━━━━━━━━━━━━━━ 27s 434ms/step
Top-1 Accuracy of squeezenet_0.375 0.7973873
Top-5 Accuracy of squeezenet_0.375 0.9772029


percentage of 3x3 filters is  0.5
61/61 ━━━━━━━━━━━━━━━━━━━━ 29s 471ms/step
Top-1 Accuracy of squeezenet_0.5 0.78739756
Top-5 Accuracy of squeezenet_0.5 0.977459


percentage of 3x3 filters is  0.625
61/61 ━━━━━━━━━━━━━━━━━━━━ 32s 521ms/step
Top-1 Accuracy of squeezenet_0.625 0.8048156
Top-5 Accuracy of squeezenet_0.625